# constellation_line_csv
星座線の始点、終点の位置をCSVファイルに保存する。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import struct

In [2]:
hip = pd.read_table("BrowseTargets.13041.1545121565", sep="|", header=2, usecols=range(1, 7), skipinitialspace=True)
hip.head()

,name,parallax,vmag,ra_deg,dec_deg,bv_color
0,HIP 48752,4.17,7.84,149.167261,-89.782454,0.097
1,HIP 71348,2.08,6.82,218.878316,-89.771696,1.698
2,HIP 42708,2.39,7.20,130.518546,-89.460502,1.712
3,HIP 78866,11.95,7.83,241.481011,-89.308449,0.412
4,HIP 22645,4.42,8.86,73.084128,-89.273961,0.260


# 星座線データの読み込み
星座線の始点・終点となる星のHIP IDが入っているCSVファイルを読み込む。  
ソースはこちら↓  
[星座線データ - Astro Commons](http://astronomy.webcrow.jp/hip/#line)  

In [3]:
line = pd.read_csv("hip_constellation_line.csv", names=("name", "HIP_S", "HIP_E"))
line.head()

,name,HIP_S,HIP_E
0,And,677,3092
1,And,3092,5447
2,And,9640,5447
3,And,5447,4436
4,And,4436,3881


# 欠損値処理
各項目の欠損値の数

In [4]:
hip.isnull().sum()

name           0
parallax     263
vmag           1
ra_deg       263
dec_deg      263
bv_color    1281
dtype: int64

Parallaxが0以下の星は除外する。

In [5]:
hip = hip[(hip["parallax"] > 0)]

bv_colorはなんか適当に埋めてみる。  
とりあえず中央値で埋める。

In [6]:
hip["bv_color"] = hip["bv_color"].fillna(hip["bv_color"].median())

欠けたデータを除外した後の星の数

In [7]:
len(hip)

113711

# 座標変換
年周視差([parallax](https://heasarc.gsfc.nasa.gov/W3Browse/star-catalog/hipparcos.html#parallax)) $pi$と距離(parsec) $D$の関係は$D = 1000 / pi$。1 parsecは3.26光年。  
球座標系を直交座標系に変換する式はこんな感じになるはず。

$$
\begin{eqnarray}
x &=& \dfrac {1000}{P_{parallax}}\cos \theta_{dec} \cos \phi_{ra} \\
y &=& \dfrac {1000}{P_{parallax}}\cos \theta_{dec} \sin \phi_{ra} \\
z &=& \dfrac {1000}{P_{parallax}}\sin \theta_{dec}
\end{eqnarray}
$$

In [8]:
def radec2xyz(star):
    x = 1000 / star["parallax"] * np.cos(np.deg2rad(star["dec_deg"])) * np.cos(np.deg2rad(star["ra_deg"]))
    y = 1000 / star["parallax"] * np.cos(np.deg2rad(star["dec_deg"])) * np.sin(np.deg2rad(star["ra_deg"]))
    z = 1000 / star["parallax"] * np.sin(np.deg2rad(star["dec_deg"]))
    
    # Unityはyが上、zが前なので順序を変更
    return [x, z, y]

# CSVファイルの作成
星座線の始点、終点となる星のIDを順に読み出し、星の位置を計算してリストを作成する。

In [9]:
constellation_line = pd.DataFrame({"xs": [],
                   "ys": [],
                   "zs": [],
                   "xe": [],
                   "ye": [],
                   "ze": [],
                   "name": []})

for index, i in line.iterrows():
    try:
        start_pos = radec2xyz(hip[hip["name"].str.startswith("HIP "+str(i["HIP_S"])+" ")].iloc[0])
    except IndexError:
        start_pos = radec2xyz(hip[hip["name"].str.startswith("HIP "+str(i["HIP_S"]))].iloc[0])   
    try:
        end_pos = radec2xyz(hip[hip["name"].str.startswith("HIP "+str(i["HIP_E"])+" ")].iloc[0])
    except IndexError:
        end_pos = radec2xyz(hip[hip["name"].str.startswith("HIP "+str(i["HIP_E"]))].iloc[0])
    name = [i["name"]]
    constellation_line = constellation_line.append(pd.Series(start_pos+end_pos+name, index=constellation_line.columns), ignore_index=True)

constellation_line.to_csv("constellation_line.csv",header=False, index=False)